In [16]:
import pandas as pd

These files were collected from the cluster using the following commands, respectively: 

    (base) arh234@g2-login-01:~/job_logs$ sacct -PT --delimiter=, --long --starttime 2022-03-27 > JobsLongSummary2022-03-27.csv
    (base) arh234@g2-login-01:~/job_logs$ sacct --format=JobID,Jobname,Partition,state,submit,start,end,elapsed,ReqMem,MaxRss,MaxVMSize,MaxPages,TotalCPU,NodeList 
    -PT --delimiter=, --starttime 2022-03-27 > JobsSummary2022-03-27.psv

---
*These commands are now aliased in `~/.bash_aliases` (see [gist notes](https://gist.githubusercontent.com/AndreaTheLinguist/fe6575ed67caa0855e7812d72cc1d08d/raw/80e58c6fa2801c62f81fc5f20037e1a103add3d1/Notes_03-27-22.md) for info on all `SLURM` aliases) and can be used with any chosen filter (e.g. `--starttime` as above, `-j`/`--jobid=[job ID]`, `--name=[job name]`, etc.) and output path*

In [17]:
# extra_info = pd.read_csv('/home/arh234/puddin/g2_transfers/JobsLongSummary2022-03-27.psv', sep='|')
info = pd.read_csv('JobsSummary2022-03-27.csv')
timestamps = ['Submit','Start', 'End']
info.loc[:,timestamps] = info[timestamps].apply(pd.to_datetime)
timedeltas = ['Elapsed', 'TotalCPU']


def norm_timedeltas(tds):
    # cx = '00:07.95'
    # cy = '00:00:00'
    # cz = '00.02340'    
    prepend=['00:00:','00:','']
    # pd.to_timedelta((prepend[c.count(':')] + c for c in [cx, cy, cz]))
    try: 
        update = pd.to_timedelta((prepend[c.count(':')] + c for c in tds))
    except AttributeError:
        update = tds
    
    update = update.round('s')
    
    return update
info.loc[:,timedeltas] = info[timedeltas].apply(norm_timedeltas)

# info[timestamps+timedeltas]

In [18]:
info

,JobID,JobName,Partition,State,Submit,Start,End,Elapsed,ReqMem,MaxRSS,MaxVMSize,MaxPages,TotalCPU,NodeList
0,459874_29,PuddinPcc,default_partition,NODE_FAIL,2022-03-26 23:29:19,2022-03-27 00:00:00,2022-03-27 00:18:34,0 days 00:18:34,100Gn,NaN,NaN,NaN,0 days 00:00:00,nikola-compute-03
1,459874_29.batch,batch,NaN,CANCELLED,2022-03-27 00:00:00,2022-03-27 00:00:00,2022-03-27 00:18:34,0 days 00:18:34,100Gn,NaN,NaN,NaN,0 days 00:00:00,nikola-compute-03
2,459874_29.extern,extern,NaN,CANCELLED,2022-03-27 00:00:00,2022-03-27 00:00:00,2022-03-27 00:18:34,0 days 00:18:34,100Gn,NaN,NaN,NaN,0 days 00:00:00,nikola-compute-03
3,459921_29,PuddinPcc,gpu,COMPLETED,2022-03-27 07:39:11,2022-03-27 07:39:14,2022-03-27 07:39:42,0 days 00:00:28,60Gn,NaN,NaN,NaN,0 days 00:00:08,hariharan-compute-02
4,459921_29.batch,batch,NaN,COMPLETED,2022-03-27 07:39:14,2022-03-27 07:39:14,2022-03-27 07:39:42,0 days 00:00:28,60Gn,1256K,80248K,0.0,0 days 00:00:08,hariharan-compute-02
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
115,460210_29.batch,batch,NaN,COMPLETED,2022-03-27 10:42:55,2022-03-27 10:42:55,2022-03-27 10:43:05,0 days 00:00:10,60Gn,1240K,80248K,0.0,0 days 00:00:05,sablab-gpu-04
116,460210_29.extern,extern,NaN,COMPLETED,2022-03-27 10:42:55,2022-03-27 10:42:55,2022-03-27 10:43:05,0 days 00:00:10,60Gn,1140K,80188K,0.0,0 days 00:00:00,sablab-gpu-04
117,460355,bash,default_partition-interactive,FAILED,2022-03-27 12:19:28,2022-03-27 12:19:28,2022-03-27 12:28:30,0 days 00:09:02,1000Mc,NaN,NaN,NaN,0 days 00:00:01,joachims-cpu-01
118,460355.extern,extern,NaN,COMPLETED,2022-03-27 12:19:28,2022-03-27 12:19:28,2022-03-27 12:28:30,0 days 00:09:02,1000Mc,1168K,80212K,0.0,0 days 00:00:00,joachims-cpu-01


In [19]:
puddin_output_info = pd.read_csv('master_all-slices_index.csv')
puddin_output_info

,slice_name,total_texts,first_text_id,last_text_id,tmp_slice_path,final_slice_path,conllu_path,origin_filepath,data_origin_group,final_df_path,exclusions_path,slice_number,started_at,finished_at,parsing_time
0,test_1,9999,pcc_eng_test_1.0001_x00002,pcc_eng_test_1.9999_x16049,pile_tables/slices/PccTe/tmp/pile_test-1_Pile-...,pile_tables/slices/PccTe/pile_test-1_Pile-CC_d...,PccTe.conll/pcc_eng_test-01.conllu,/share/compling/data/pile/test.jsonl,test,pile_tables/pile_test_Pile-CC_df.pkl.gz,pile_exclusions/pile_test_Pile-CC_excl.pkl.gz,1,Wed Mar 23 21:41:12 2022,Wed Mar 23 22:27:19 2022,0 days 00:46:08
1,val_1,9999,pcc_eng_val_1.0001_x00001,pcc_eng_val_1.9999_x16202,pile_tables/slices/PccVa/tmp/pile_val-1_Pile-C...,pile_tables/slices/PccVa/pile_val-1_Pile-CC_df...,PccVa.conll/pcc_eng_val-01.conllu,/share/compling/data/pile/val.jsonl,val,pile_tables/pile_val_Pile-CC_df.pkl.gz,pile_exclusions/pile_val_Pile-CC_excl.pkl.gz,1,Wed Mar 23 21:41:29 2022,Wed Mar 23 22:35:15 2022,0 days 00:53:46
2,test_2,11349,pcc_eng_test_2.00001_x16050,pcc_eng_test_2.11349_x34443,pile_tables/slices/PccTe/tmp/pile_test-2_Pile-...,pile_tables/slices/PccTe/pile_test-2_Pile-CC_d...,PccTe.conll/pcc_eng_test-02.conllu,/share/compling/data/pile/test.jsonl,test,pile_tables/pile_test_Pile-CC_df.pkl.gz,pile_exclusions/pile_test_Pile-CC_excl.pkl.gz,2,Wed Mar 23 22:27:19 2022,Wed Mar 23 23:19:57 2022,0 days 00:52:37
3,val_2,11306,pcc_eng_val_2.00001_x16204,pcc_eng_val_2.11306_x34544,pile_tables/slices/PccVa/tmp/pile_val-2_Pile-C...,pile_tables/slices/PccVa/pile_val-2_Pile-CC_df...,PccVa.conll/pcc_eng_val-02.conllu,/share/compling/data/pile/val.jsonl,val,pile_tables/pile_val_Pile-CC_df.pkl.gz,pile_exclusions/pile_val_Pile-CC_excl.pkl.gz,2,Wed Mar 23 22:35:15 2022,Wed Mar 23 23:36:09 2022,0 days 01:00:54
4,test_3,11350,pcc_eng_test_3.00001_x34445,pcc_eng_test_3.11350_x52789,pile_tables/slices/PccTe/tmp/pile_test-3_Pile-...,pile_tables/slices/PccTe/pile_test-3_Pile-CC_d...,PccTe.conll/pcc_eng_test-03.conllu,/share/compling/data/pile/test.jsonl,test,pile_tables/pile_test_Pile-CC_df.pkl.gz,pile_exclusions/pile_test_Pile-CC_excl.pkl.gz,3,Wed Mar 23 23:19:57 2022,Thu Mar 24 00:11:52 2022,0 days 00:51:56
5,val_3,11307,pcc_eng_val_3.00001_x34545,pcc_eng_val_3.11307_x52791,pile_tables/slices/PccVa/tmp/pile_val-3_Pile-C...,pile_tables/slices/PccVa/pile_val-3_Pile-CC_df...,PccVa.conll/pcc_eng_val-03.conllu,/share/compling/data/pile/val.jsonl,val,pile_tables/pile_val_Pile-CC_df.pkl.gz,pile_exclusions/pile_val_Pile-CC_excl.pkl.gz,3,Wed Mar 23 23:36:09 2022,Thu Mar 24 00:35:00 2022,0 days 00:58:52
6,29_001,9999,pcc_eng_29_001.0001_x0000001,pcc_eng_29_001.9999_x0016002,pile_tables/slices/Pcc29/tmp/pile_29-001_Pile-...,pile_tables/slices/Pcc29/pile_29-001_Pile-CC_d...,Pcc29.conll/pcc_eng_29-001.conllu,/share/compling/data/pile/train/29.jsonl,29,pile_tables/pile_29_Pile-CC_df.pkl.gz,pile_exclusions/pile_29_Pile-CC_excl.pkl.gz,1,Sun Mar 27 02:16:40 2022,Sun Mar 27 03:02:53 2022,0 days 00:46:13
7,29_002,9999,pcc_eng_29_002.0001_x0016004,pcc_eng_29_002.9999_x0032274,pile_tables/slices/Pcc29/tmp/pile_29-002_Pile-...,pile_tables/slices/Pcc29/pile_29-002_Pile-CC_d...,Pcc29.conll/pcc_eng_29-002.conllu,/share/compling/data/pile/train/29.jsonl,29,pile_tables/pile_29_Pile-CC_df.pkl.gz,pile_exclusions/pile_29_Pile-CC_excl.pkl.gz,2,Sun Mar 27 03:02:53 2022,Sun Mar 27 03:51:05 2022,0 days 00:48:11
8,29_003,9999,pcc_eng_29_003.0001_x0032275,pcc_eng_29_003.9999_x0048551,pile_tables/slices/Pcc29/tmp/pile_29-003_Pile-...,pile_tables/slices/Pcc29/pile_29-003_Pile-CC_d...,Pcc29.conll/pcc_eng_29-003.conllu,/share/compling/data/pile/train/29.jsonl,29,pile_tables/pile_29_Pile-CC_df.pkl.gz,pile_exclusions/pile_29_Pile-CC_excl.pkl.gz,3,Sun Mar 27 03:51:05 2022,Sun Mar 27 04:36:03 2022,0 days 00:44:57
9,29_004,9999,pcc_eng_29_004.0001_x0048553,pcc_eng_29_004.9999_x0064721,pile_tables/slices/Pcc29/tmp/pile_29-004_Pile-...,pile_tables/slices/Pcc29/pile_29-004_Pile-CC_d...,Pcc29.conll/pcc_eng_29-004.conllu,/sha